In [1]:
#!pip install datetime
#!pip install numpy
#!pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
import datetime

In [10]:
#df = pd.read_csv('agg3.csv')
df = pd.read_csv('aggtest.csv')

In [11]:
## is the transaction made in the home city
df['in_home_city'] = (df['HomeCity'] == df['City'])

## how many times have they previously made a transaction in this city
df['Timestamp_x'] = pd.to_datetime(df['Timestamp_x'], format='ISO8601')
df = df.sort_values('Timestamp_x')
df['prev_trans_in_city'] = df.groupby(['City', 'CustomerID']).cumcount()

## How many times has the user previously used this device
df['times_device_used'] = df.groupby(['CustomerID', 'DeviceID']).cumcount()

## How many devices has the user used in the past
df = df.sort_values(['CustomerID', 'Timestamp_x'])
def cumulative_unique_devices(g):
    seen = set()
    counts = []
    for device in g['DeviceID']:
        counts.append(len(seen))
        seen.add(device)
    return pd.Series(counts, index=g.index)
df['unique_devices_past'] = df.groupby('CustomerID', group_keys=False).apply(cumulative_unique_devices, include_groups=False)
idx = df['Online'] == 0
df.loc[idx, 'unique_devices_past'] = np.nan

In [12]:
#Number of days since account opened
df['OpenDate'] = pd.to_datetime(df['OpenDate'])
df['Timestamp_dt'] = pd.to_datetime(df['Timestamp_dt'])
df["days_since_open"] = (df["Timestamp_dt"] - df["OpenDate"]).dt.days
#Number of days since account opened
df['OpenDate_dst'] = pd.to_datetime(df['OpenDate_dst'])
df["days_since_open_dst"] = (df["Timestamp_dt"] - df["OpenDate_dst"]).dt.days

#Time since Xact for each customer
df['Timestamp_dt'] = pd.to_datetime(df['Timestamp_dt'],format="%Y/%m/%d")
df = df.sort_values(["CustomerID", "Timestamp_dt"])

df["prev_Timestamp"] = df.groupby("CustomerID")["Timestamp_dt"].shift(1)
df["time_since_last_txn_days"] = (df["Timestamp_dt"] - df["prev_Timestamp"]).dt.days.fillna(0).astype(int)
df["time_since_last_txn_hours"] = ((df["Timestamp_dt"] - df["prev_Timestamp"]).dt.total_seconds() / 3600).fillna(0).astype(int)

#Xacts per day for each customer
df['Timestamp_dt'] = pd.to_datetime(df['Timestamp_dt'],format="%Y/%m/%d")
df = df.sort_values(["CustomerID", "Timestamp_dt"])
df["day"] = df["Timestamp_dt"].dt.floor("D")
df["xacts_earlier_today"] = (
    df.groupby(["CustomerID", "day"])
      .cumcount()
)

#Amount deviation
df['Timestamp_dt'] = pd.to_datetime(df['Timestamp_dt'],format="%Y/%m/%d %H:%M")
df = df.sort_values(["CustomerID", "Timestamp_dt"])
df["avg_amount_past"] = (df.groupby("CustomerID")["Amount"]).transform(lambda x: x.shift(1).rolling(10, min_periods=1).mean())

#Measure direction, >1 spending more than usual <1 less
df["amount_ratio"] = df["Amount"]/df["avg_amount_past"]

#Deviation Mesaure distance from mean, regardless of direction. 0 normal higher means unusual
df["amount_dev"] = (abs(df["Amount"] - df["avg_amount_past"]))/df["avg_amount_past"]

#MCC group

# df = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_train.csv", parse_dates=["Timestamp_dt"])
df['Timestamp_dt'] = pd.to_datetime(df['Timestamp_dt'],format="%Y/%m/%d %H:%M")
df = df.sort_values(["CustomerID", "Timestamp_dt"])
df["mcc_group_prev_count"] = (df.groupby(["CustomerID", "MCC_Group"]).cumcount())

In [13]:
df

,Unnamed: 0,Timestamp_dt,TxnID,Timestamp_x,Online,CustomerID,SrcAccount,DstAccount,Channel,MCC_Group,...,days_since_open_dst,prev_Timestamp,time_since_last_txn_days,time_since_last_txn_hours,day,xacts_earlier_today,avg_amount_past,amount_ratio,amount_dev,mcc_group_prev_count
323,323,2025-01-01,c77cfd37-2a4f-41a2-ae45-696aaa2ec4e9,2025-01-01 09:07:00,0,C000001,Aa95695be,NaN,ATM,Groceries,...,NaN,NaT,0,0,2025-01-01,0,NaN,NaN,NaN,0
550,550,2025-01-01,1e3da79d-891c-4769-af85-4b9bfc2aab7c,2025-01-01 16:09:00,0,C000001,Aa95695be,NaN,branch,Utilities,...,NaN,2025-01-01,0,0,2025-01-01,1,26.436533,1.494917,0.494917,0
1443,1443,2025-01-02,b1e5643f-3fee-41e6-b003-a360810dfa3d,2025-01-02 17:15:00,0,C000001,Aa95695be,NaN,branch,Restaurants,...,NaN,2025-01-01,1,24,2025-01-02,0,32.978478,2.510326,1.510326,0
2332,2332,2025-01-03,540ab70c-c05e-4a68-bddb-fb176e08bf62,2025-01-03 18:47:00,0,C000001,Aa95695be,NaN,branch,Groceries,...,NaN,2025-01-02,1,24,2025-01-03,0,49.581226,0.247022,0.752978,1
3327,3327,2025-01-04,2e1ccb91-e11f-46a5-886f-6f9a23550d0a,2025-01-04 22:52:47,1,C000001,Aa95695be,A0c70a82e,online,Utilities,...,209.0,2025-01-03,1,24,2025-01-04,0,40.247838,-1.617362,2.617362,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148908,148908,2025-06-27,743db2e8-34dd-41ce-8856-fa47df3285d7,2025-06-27 17:03:00,0,C000998,A651ebefd,NaN,branch,Utilities,...,NaN,2025-06-27,0,0,2025-06-27,1,39.981647,2.266272,1.266272,25
149911,149911,2025-06-28,0bca8ddb-3a15-40b2-884f-a1aa22dc4645,2025-06-28 22:08:20,1,C000998,A651ebefd,Ae57cca93,online,Online Retail,...,1745.0,2025-06-27,1,24,2025-06-28,0,44.429436,-0.967026,1.967026,59
149912,149912,2025-06-28,03abc94f-9c96-4d57-beb8-f4cd58f31478,2025-06-28 22:08:20,1,C000998,A651ebefd,NaN,online,Restaurants,...,NaN,2025-06-28,0,0,2025-06-28,1,34.045957,-0.571354,1.571354,55
150417,150417,2025-06-29,f31540d5-a955-42dc-bd7e-7e441e878104,2025-06-29 12:28:00,0,C000998,A651ebefd,NaN,branch,Restaurants,...,NaN,2025-06-28,1,24,2025-06-29,0,29.586858,2.761952,1.761952,56


In [6]:
## Save to csv
# df.to_csv('processed_data.csv')
df.to_csv('processed_data_test.csv')